Identification of Autism using magnetic Resonance Imaging A novel approach combining Convolutional Neural Network-Ensemble learning (CNN-EL) Compared with PCA+ SVM APPROACH FOR ACCURACY IMPROVEMENT (7%).

## PCA+ SVM

In [20]:
from sklearn.decomposition import PCA

In [21]:
import numpy as np
import pandas as pd
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model,Sequential,load_model,Input

from keras.applications import MobileNet,MobileNetV2,InceptionV3
from keras.applications.mobilenet_v2 import preprocess_input
from keras.layers import AveragePooling2D,Flatten,Dense,Dropout,MaxPooling2D
from keras.optimizers import Adam

from keras.preprocessing.image import img_to_array,load_img

from sklearn.metrics import confusion_matrix,classification_report



import cv2

In [22]:
import matplotlib.pyplot as plt

In [23]:
os.listdir('train')

['autistic', 'non_autistic']

In [5]:
data=[]
label=[]
for i in ['train','test']:
    for j in os.listdir(i):
        for z in os.listdir(f'{i}/{j}'):
            img=load_img(f'{i}/{j}/{z}',target_size=(64,64))
            img1=img_to_array(img)
            data.append(img1.flatten())
            label.append(j)
            
            

In [6]:
x=np.array(data)
y=np.array(label)

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=7)

In [8]:
# Dimensionality reduction - Full PCA
im_pca = PCA()
im_pca.fit(x_train)
variance_explained_list = im_pca.explained_variance_ratio_.cumsum()
print(variance_explained_list)

[0.14496319 0.24828118 0.33124638 ... 0.99999934 1.0000008  1.0000008 ]


In [9]:
test_x_pca = im_pca.transform(x_test)
train_x_pca = im_pca.transform(x_train)

In [10]:
#from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
svm=SVC()
svm.fit(train_x_pca,y_train)

SVC()

In [11]:
print('The Train Score',svm.score(train_x_pca,y_train))
print('The Test Score',svm.score(test_x_pca,y_test))

The Train Score 0.9351198871650211
The Test Score 0.7362482369534555


# CNN

In [12]:
datagen=ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,
                             horizontal_flip=True,fill_mode="nearest")

train_data=datagen.flow_from_directory('train/',target_size=(299,299),class_mode="categorical",batch_size=64)


test_data=datagen.flow_from_directory('test/',target_size=(299,299),class_mode="categorical",batch_size=64)

Found 2536 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In [13]:
train_data.class_indices

{'autistic': 0, 'non_autistic': 1}

In [14]:
bot_model=InceptionV3(weights='imagenet',include_top=False,input_tensor=Input(shape=(299,299,3)))

top_model=bot_model.output
top_model=MaxPooling2D(pool_size=(7,7))(top_model)
top_model=Flatten()(top_model)
top_model = Dense(128, activation="relu")(top_model)
top_model = Dropout(0.5)(top_model)
top_model = Dense(2, activation="softmax")(top_model)

In [15]:
fin_model = Model(inputs=bot_model.input, outputs=top_model)

In [16]:
for layer in bot_model.layers:
    layer.trainable = False

In [17]:
opti = Adam(lr=0.0001, decay=0.0001 /20)
fin_model.compile(loss="categorical_crossentropy", optimizer=opti,metrics=["accuracy"])

In [18]:
history = fin_model.fit_generator(train_data,steps_per_epoch=3066 //32,
                    validation_data=test_data,validation_steps=767//20,
                    epochs=20)

C:\Users\HP\Anaconda3\envs\projectmain\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
95/95 [==============================] - 187s 2s/step - loss: 23.3034 - accuracy: 0.5254 - val_loss: 5.9311 - val_accuracy: 0.6133


In [19]:
names=np.array(['autistic','non_autistic'])
image=load_img('test/non_autistic/126.jpg',target_size=(299,299))
img=img_to_array(image)
ex_im=np.expand_dims(img,axis=0)
class2=fin_model.predict(ex_im)
names[np.argmax(class2)]

'autistic'